# SQLalchemy
database->Python

pip install sqlalchemy

In [18]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from datetime import datetime

In [19]:
Base= declarative_base()
class Task(Base):
    __tablename__ = 'tasks'
    id = Column(Integer, primary_key=True)
    task=Column(String)
    deadline = Column(DateTime, default=datetime.now())



In [43]:
engine= create_engine('sqlite:///mydb.sqlite3',echo=True)
Base.metadata.create_all(engine)

2024-08-16 05:49:12,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 05:49:12,095 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("tasks")
2024-08-16 05:49:12,098 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-16 05:49:12,103 INFO sqlalchemy.engine.Engine COMMIT


In [45]:
def open_db():
    engine= create_engine('sqlite:///mydb.sqlite3')
    Sesssion= sessionmaker(bind=engine)
    return Sesssion()

ORM

In [22]:
db=open_db()
db.add(Task(task="Learn SQLAlchemy", deadline=datetime(2024,8,1)))
db.add(Task(task="Happy Independence Day"))
db.add(Task(task="Take out the trash"))
db.add(Task(task="Clear the fridge",deadline=datetime(2024,8,23)))
db.add(Task(task="Generate the report",deadline=datetime(2024,8,16)))
db.add(Task(task="Pay Subcription fees",deadline=datetime(2024,8,18)))
db.add(Task(task="Celebrate birthday",deadline=datetime(2024,8,25)))


db.commit()
db.close()
       

2024-08-16 05:40:22,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 05:40:22,879 INFO sqlalchemy.engine.Engine INSERT INTO tasks (task, deadline) VALUES (?, ?)
2024-08-16 05:40:22,881 INFO sqlalchemy.engine.Engine [generated in 0.00240s] ('Learn SQLAlchemy', '2024-08-01 00:00:00.000000')
2024-08-16 05:40:22,890 INFO sqlalchemy.engine.Engine INSERT INTO tasks (task, deadline) VALUES (?, ?) RETURNING id
2024-08-16 05:40:22,893 INFO sqlalchemy.engine.Engine [generated in 0.00031s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Happy Independence Day', '2024-08-16 05:40:22.768317')
2024-08-16 05:40:22,896 INFO sqlalchemy.engine.Engine INSERT INTO tasks (task, deadline) VALUES (?, ?) RETURNING id
2024-08-16 05:40:22,901 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Take out the trash', '2024-08-16 05:40:22.768317')
2024-08-16 05:40:22,906 INFO sqlalchemy.engine.Engine INSERT INTO tasks (task, deadline) VALUES (?, ?) RETURNING id

Querying the database
- get all the data
- get specific data using filters

In [23]:
db= open_db()
tasks= db.query(Task).all()
for item in tasks:
    print(f'# {item.id} | {item.task:30} | {item.deadline.date()} | {item.deadline.time()}')
db.close()

2024-08-16 05:40:22,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 05:40:22,980 INFO sqlalchemy.engine.Engine SELECT tasks.id AS tasks_id, tasks.task AS tasks_task, tasks.deadline AS tasks_deadline 
FROM tasks
2024-08-16 05:40:22,982 INFO sqlalchemy.engine.Engine [generated in 0.00225s] ()
# 1 | Learn SQLAlchemy               | 2024-08-01 | 00:00:00
# 2 | Happy Independence Day         | 2024-08-16 | 05:37:36.184176
# 3 | Take out the trash             | 2024-08-16 | 05:37:36.184176
# 4 | Clear the fridge               | 2024-08-23 | 00:00:00
# 5 | Generate the report            | 2024-08-16 | 00:00:00
# 6 | Pay Subcription fees           | 2024-08-18 | 00:00:00
# 7 | Celebrate birthday             | 2024-08-25 | 00:00:00
# 8 | Learn SQLAlchemy               | 2024-08-01 | 00:00:00
# 9 | Happy Independence Day         | 2024-08-16 | 05:40:22.768317
# 10 | Take out the trash             | 2024-08-16 | 05:40:22.768317
# 11 | Clear the fridge               | 2024-08-23 | 00

In [33]:
#get item by id
id=6
db=open_db()
task=db.get(Task, id)
if task:
    print(f'# {item.id} | {item.task:30} | {item.deadline.date()} | {item.deadline.time()}')



2024-08-16 05:40:51,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 05:40:51,590 INFO sqlalchemy.engine.Engine SELECT tasks.id AS tasks_id, tasks.task AS tasks_task, tasks.deadline AS tasks_deadline 
FROM tasks 
WHERE tasks.id = ?
2024-08-16 05:40:51,593 INFO sqlalchemy.engine.Engine [generated in 0.00281s] (6,)
# 14 | Celebrate birthday             | 2024-08-25 | 00:00:00


In [36]:
title='Take out the trash'
db=open_db()
task=db.query(Task).filter(Task.task==title).all()
for item in task:
    print(f'# {item.id} | {item.task:30} | {item.deadline.date()} | {item.deadline.time()}')

db.close()

2024-08-16 05:46:48,914 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 05:46:48,919 INFO sqlalchemy.engine.Engine SELECT tasks.id AS tasks_id, tasks.task AS tasks_task, tasks.deadline AS tasks_deadline 
FROM tasks 
WHERE tasks.task = ?
2024-08-16 05:46:48,921 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ('Take out the trash',)
# 3 | Take out the trash             | 2024-08-16 | 05:37:36.184176
2024-08-16 05:46:48,928 INFO sqlalchemy.engine.Engine ROLLBACK


In [46]:
#filter items by date
date="16-08-2024"
date=datetime.strptime(date, "%d-%m-%Y")
db=open_db()
tasks=db.query(Task).filter(Task.deadline==date).all()
for item in tasks:
    print(f'# {item.id} | {item.task:30} | {item.deadline.date()} | {item.deadline.time()}')
db.close()

# 5 | Generate the report            | 2024-08-16 | 00:00:00


In [47]:
db=open_db()
tasks=db.query(Task).filter(Task.task.icontains("day")).all()
for item in tasks:
    print(f'# {item.id} | {item.task:30} | {item.deadline.date()} | {item.deadline.time()}')

# 2 | Happy Independence Day         | 2024-08-16 | 05:37:36.184176
# 7 | Celebrate birthday             | 2024-08-25 | 00:00:00


In [50]:
#update item
id= 7
db=open_db()
result = db.get(Task,id)
if result:
    result.task="Generate student report"
    result.deadline=datetime(2024,8,20)
    db.commit()
db.close()

In [51]:
#delete item
id= 6
db=open_db()
result = db.get(Task,id)
if result:
    db.delete="Generate student report"
    db.commit()
    print("Item deleted successfully")
db.close()

Item deleted successfully
